In [ ]:
# Build the input files
swash_wrapper.build_cases(mode="one_by_one")
swash_wrapper.set_cases_dirs_from_output_dir()
# Run the model using docker_serial launcher
swash_wrapper.run_cases(launcher="nohup launchSwash.sh", num_workers=30)

swash_wrapper.run_cases_bulk(launcher="sbatch --array=0-799 /home/grupos/geocean/valvanuz/HySwash/SlurmChy.sh")

failed_runs=swash_wrapper.monitor_cases()


Modify nodes_per_wavelength and rerun failed cases

In [ ]:
failedRuns=[20,30,34,36,42,64,90,94,99,110,125,153,170,210,248,270,282,362,403,440,483,497,499,506,517,527,567,578,580,591,607,626,632,677,699,714,724,725,762,765]
swash_wrapper.build_cases(mode="one_by_one",cases_to_build=failedRuns)
swash_wrapper.run_cases_bulk(launcher="sbatch --array=20,30,34,36,42,64,90,94,99,110,125,153,170,210,248,270,282,362,403,440,483,497,499,506,517,527,567,578,580,591,607,626,632,677,699,714,724,725,762,765 /home/grupos/geocean/valvanuz/HySwash/SlurmChy.sh")
failedRuns2=[16,20,64,90,94,117,238,326,457,483,607,649,765,768]

Check failed runs are not correlated

In [ ]:
from copy import deepcopy
mda_failed=deepcopy(mda)
mda_failed.centroids = mda.centroids.iloc[failedRuns2]
mda_failed.plot_selected_centroids()

Remove folders whose output files do not correspond with the case_num

In [ ]:
# Remove folders whose output files do not correspond with the case_num
folders = os.listdir(output_dir)
for folder in folders:
    # Get the name of the folder
    folder_path = os.path.join(output_dir, folder)
    # Get the name of the output files
    try:
        a=xr.open_dataset(folder_path + "/output_postprocessed.nc")
        if int(a.case_num.values[0]) != int(folder):
            print("rm -f %s/*.nc"%(folder))
    except:
        pass

Check there are no Nans in postprocessed Output

In [ ]:
# Check if there are any NaN values in the postprocessed data
# For every case_num in postprocessed dataset
for case_num in postprocessed.case_num.values:
    postprocessed_case = postprocessed.sel(case_num=case_num)
    for var in postprocessed_case.data_vars:
        if postprocessed_case[var].isnull().values.any():
            # print the variable name and the first occurrence of NaN
            nan_indices = np.where(np.isnan(postprocessed_case[var].values))
            print(f"{var} contains NaN values at indices: {case_num}")

Check there are no Nans in postprocessed output

In [ ]:
# Check if there are any NaN values in the postprocessed data
# For every case_num in postprocessed dataset
for case_num in postprocessed.case_num.values:
    postprocessed_case = postprocessed.sel(case_num=case_num)
    for var in postprocessed_case.data_vars:
        if postprocessed_case[var].isnull().values.any():
            # print the variable name and the first occurrence of NaN
            nan_indices = np.where(np.isnan(postprocessed_case[var].values))
            print(f"{var} contains NaN values at indices: {case_num}")